In [1]:
import sys
sys.path.append('..')
from src import datagen
from src import engine
from src import utils
import yaml
from time import time
import os
import pandas as pd


In [2]:
table = pd.read_csv('../data/TADPOLE_D1_D2_proc_norm.csv')

In [3]:
def search_table(table,pid):
    '''
    Searches the table for the entries corresponding to the pid. Checks if pid label is 'Dementia'.
    
    Returns True is 'Dementia'
    Returns False if not
    '''
    label = table[(table['PTID'] == pid) & (table['VISCODE'] =='bl')]['DX'].values[0]
    if label == 'Dementia' or label == 'AD':
        return True
    else:
        return False

In [4]:
def pid_convert(pid):
    '''
    Converts pids that in the form of integers to strings in the proper format
    
    Example: 012368 --> 001_S_2368
    
             12345  --> 001_S_2345
    '''
    p = pid.item()
    if 100000 < p < 1000000:
        p = '0'+str(p)[:2] + '_S_' + str(p)[2:]
    elif p < 100000 :
        p = '00' + str(p)[:1] + '_S_' + str(p)[1:]
    else:
        p = str(p)[:3] + '_S_' + str(p)[3:]
    return p

class Trajectory_Stats:
    def __init__(self, pid, y, y_pred, trajectory_id):
        self.pid = pid
        self.y = y
        _, self.y_pred = y_pred.max(0)
        
        self.trajectory_id = trajectory_id
#class Trajectories_Stats
#    def __init__(self,pid,trajectories)

class Cascade:
    def __init__(self, trajectories):
        '''
        A cascade is a series of trajectories from a single patient used for tabulating results for
        AD transition detection at the patient level.
        '''
        self.trajectories = trajectories
      
        flag = 0
        
        # Record the index where the transition occurs in ground truth
        for t in trajectories:
            if(t.y == 2):
                flag = 1
                self.transition_true = t.trajectory_id[-1]
                break
                
        if flag == 0:
            self.transition_true = -1
            
        flag = 0
        
        # Record the index where the transition occurs in FLARe
        for t in trajectories:
            if(t.y_pred == 2):
                flag = 1
                self.transition_flare = t.trajectory_id[-1]
                break
                
        if flag == 0:
            self.transition_flare = -1
           
        # Check for false positives. If transition true = -1, then the patient never develops AD
        if(self.transition_true == -1):
            self.type = 'No AD'
            if(self.transition_flare != -1):
                self.FP = True
            else:
                self.FP = False
        else:
            self.type = 'AD'
            if(self.transition_flare == -1):
                self.FN = True
            else:
                self.FN = False
                self.diff = self.transition_flare - self.transition_true
        
            
        

In [5]:
with open('../data/stats.pickle','rb') as f:
    output = pickle.load(f)
print(len(output[0]['y']))

3489


In [13]:
def gen_patient_dict(val_dict_T):
    patients = {}
    num_entries = len(val_dict_T['pid'])
    with open('../data/pids.pickle','rb') as f:
        unique_pids = pickle.load(f)
    patients = {}
    
    # initialize patient dict
    for x in unique_pids:
        patients[x] = []
    
    for i in range(num_entries):
        pid = pid_convert(val_dict_T['pid'][i])
        temp_traj = Trajectory_Stats(pid, val_dict_T['y'][i],val_dict_T['y_pred'][i],val_dict_T['trajectory_id'][i])
        patients[pid].append(temp_traj)
    return patients



In [12]:
def create_cascade(patients, T):
    '''
    Takes a patient dict as input and creates cascades for a certain value of 'T'
    '''
    final = {}
    for key in patients.keys():
        # sort Trajectory objects by lexicographical order of their trajectory id's
        patients[key].sort(key=lambda k: k.trajectory_id.tolist())
        final[key] = []
        for t in patients[key]:
            if(T == 1):
                if t.trajectory_id[0] == 0:
                    final[key].append(t)
            elif (T == 2):
                if t.trajectory_id[0] == 0 and t.trajectory_id[1] == 1:
                    final[key].append(t)
         
    for key in final.keys():
        # If the patient doesn't already have AD (didn't develop within first visit)
        #if final[key][0].y != 2:
        cascades[key] = Cascade(final[key])
    return cascades

In [15]:
def print_counts(cascades):
    '''
    Takes a dict of cascades as input and prints out counts
    '''
    counts = [0]*8
    FN = 0
    FP = 0
    total_no_ad = 0
    first_visit = 0
    developed = 0
    for key in cascades.keys():
        cascade = cascades[key]
        if cascade.type == 'AD':
            if(not cascade.FN):
                if(cascade.trajectories[0].y == 2):
                    if(not search_table(table,key)):
                        developed += 1
                        counts[cascade.diff + 3] += 1
                    else:
                        first_visit += 1
                else:
                    developed += 1
                    counts[cascade.diff + 3] += 1
            else:
                developed += 1
                FN += 1
        if cascade.type == 'No AD':
            total_no_ad += 1
            if cascade.FP:
                FP += 1
    print('Predicted within... index 2 corresponds to -1',counts)
    print('False Negatives',FN)
    print("Total people in test set who didn't develop AD",total_no_ad)
    print('... and of those people the number of False Positives',FP)
    print("Total people who already had AD in the first visit", first_visit)
    total = np.sum(counts) + FN + FP + total_no_ad + already_ad + first_visit
    print("Total people who developed AD:", developed)
    print("Total people", total)


In [16]:
patients = gen_patient_dict(output[0])
cascades = create_cascade(patients,1)
print_counts(cascades)

Predicted within... index 2 corresponds to -1 [0, 0, 18, 28, 0, 1, 1, 1]
False Negatives 7
Total people in test set who didn't develop AD 209
... and of those people the number of False Positives 7
Total people who already had AD in the first visit 66
Total people who developed AD: 56
Total people 338
